In [1]:
import requests 

In [2]:
from bs4 import BeautifulSoup 

In [4]:
import re 

In [5]:
def start_requests(url):
    r = requests.get(url)
    return r.content

In [6]:
movie_Dict = {'Movies Name':[],'Release Year':[], 'Genre':[], 'Director':[], 'Votes':[]}

In [10]:
def parse(text):
    soup = BeautifulSoup(text, 'html.parser')
    movie_list = soup.find_all('div', class_ = 'lister-item-content') # 'div class' list all info. of each movie
    for movie in movie_list:
        movies_Name = movie.find_all('a')[0].text
        movies_Year = movie.find_all('span')[1].text
        movies_ReleaseYear = re.findall(r'\d{4}', movies_Year)[0]
        movies_Genre = movie.find_all('span', class_ = 'genre')[0].text.strip()
        movies_Director = movie.find_all('p')[2].find_all('a')[0].text
        
        movie_Dict['Movies Name'].append(movies_Name)
        movie_Dict['Release Year'].append(movies_ReleaseYear)
        movie_Dict['Genre'].append(movies_Genre)
        movie_Dict['Director'].append(movies_Director)
        
    return movie_Dict


def main():
    for i in range(2):
        x = 1 + (i * 50)
        url = f'https://www.imdb.com/search/title?title_type=feature&start={x}&ref_=adv_nxt'
        text = start_requests(url)
        parse(text)

if __name__ == '__main__':
    main()

#print(movie_Dict)
